# `uv` quick reference

`uv` is an extremely fast Python package and project manager, written in Rust.

* Official docs: [https://docs.astral.sh/uv/](https://docs.astral.sh/uv/)

# Configuration

uv will search for these files in the current directory or in the nearest parent directory.

* `pyproject.toml`
*  `uv.toml`

If both files are found, then  `uv.toml` has priority and `pyproject.toml` is ignored.



# Installation

## Install `uv`

* Quick install command:
    * MacOS / Linux:
        * `curl -LsSf https://astral.sh/uv/install.sh | sh`
    * Windows:
        * `powershell -c "irm https://astral.sh/uv/install.ps1 | more"`
* Full installation guide:
    * [https://docs.astral.sh/uv/getting-started/installation/](https://docs.astral.sh/uv/getting-started/installation/)

## Check version

* `uv version` - check version of installed `uv`

# Managing Python

Detailed docs: [https://docs.astral.sh/uv/guides/install-python/](https://docs.astral.sh/uv/guides/install-python/)

**FAQ**

* **Do I need to install Python separately if I use `uv`?**
    * No, not necessarily. If Python is already installed on your system, `uv` will detect and use it without configuration. However, `uv` can also install and manage Python versions for you.
* **Why does uv install its own Python when my project already has Python in `.env`?**
    * There are two different Python installations involved:
        1. **Base Python** (`uv python install <version>`)
           - Serves as a template for creating project environments
           - Contains all core Python components needed to create virtual environments
        2. **Project Environment** (`.env`)
           - A lightweight setup that mostly links back to this base installation
           - Does not copy everything
* **Is the .env folder a full copy of Python?**
    * No. The project's `.env` folder is lightweight and uses symlinks to reference the base Python installation. It only contains:
        - Project-specific packages
        - Minimal configuration
        - Links to the base Python installation
* **Can I use Python installed by uv directly from my OS?**
    * No. Python versions installed by `uv` are:
        - Managed in uv's own directory
        - Only accessible through `uv` commands
        - Not available as system-wide Python installations
* **Why not download Python separately for each project?**
    * Having a single base installation is more efficient because it:    
        - Avoids downloading and setting up full Python repeatedly
        - Prevents duplicating hundreds of megabytes of core files
        - Maintains consistency in core Python components across projects

## Available Python versions

* `uv python list`
    * Shows Python versions available through uv, including:
        1. **Already installed versions:**
            * Lists Python versions installed by uv in its managed directory
            * Shows their exact installation paths
            * Indicates which version is currently active (if any)
        2. **Available for installation:**
            * Shows all Python versions that can be installed via uv
            * Includes release date and status (stable/beta/alpha)
            * Lists architecture-specific builds available
    * Search locations:
        * Primary: `~/.local/share/uv/python/` (Unix) or `%LOCALAPPDATA%/uv/python/` (Windows)
        * Fetches available version information from Python's official release index
    * Useful for:
        * Checking which Python versions are already installed
        * Finding new versions available for installation
        * Managing multiple Python versions in your development environment

## Install Python version

* `uv python install 3.12`
    * Installs a specific Python version for use with uv projects
    * Installation process:
        1. Downloads the official Python build for your OS/architecture
        2. Verifies the download integrity
        3. Installs into uv's managed directory:
            * Unix/MacOS: `~/.local/share/uv/python/`
            * Windows: `%LOCALAPPDATA%/uv/python/`
    * Installation options:
        * `uv python install 3.12` - installs latest 3.12.x release
        * `uv python install 3.12.1` - installs exact version
        * `uv python install --list 3.12` - shows available 3.12.x versions
        * `uv python install --reinstall 3.12` - forces reinstallation
    * What gets installed:
        * Core Python interpreter
        * Standard library
        * pip (isolated within uv's environment)
        * Basic tools (idle, venv, etc.)
    * Post-installation:
        * The installed Python is only managed by uv
        * Not available directly from system PATH
        * Used as a template for creating virtual environments
        * Can be referenced in project's `.python-version` file

## Find used Python

* `uv python find`
    * Locates Python installations that uv can use, searching in this order:
        1. The project's `.python-version` file (if it exists)
        2. Python versions installed by uv (in `~/.local/share/uv/python/` on Unix-like systems)
        3. System-wide Python installations (in PATH)
        4. Other common Python installation locations:
            * `/usr/bin/python*`
            * `/usr/local/bin/python*`
            * Windows: Python installations in the Registry
            * macOS: Framework installations
    * Returns information about the first valid Python installation found
    * Useful for:
        * Debugging which Python uv would use for a project
        * Verifying Python installation paths
        * Checking if uv can find your preferred Python version

## Uninstall Python version

* `uv python uninstall 3.12`

## Pin Python version

* `uv python pin 3.12`
    * Set specified version of Python the current project = folder.
    * This creates a `.python-version` file in your current directory.
        * This file acts like a bookmark that tells uv exactly which Python version should be used for this project.

# Initialize new project

Detailed docs: https://docs.astral.sh/uv/guides/projects/

1. `cd` into project folder
2. `uv init` - initialize project setup using `uv`
    * This command creates files:
        * `.python-version` - pinned python version for this project (folder)
        * `README.md` - introductory info about project
        * `hello.py` - first python demo file
        * `pyproject.toml` - project configuration including **base** dependencies
        * `uv.lock` - **complete** dependencies to create exact reproducible environments
            * It is human-readable TOML file but is managed by uv and should not be edited manually.
        * `.git` - initialized git repo
        * `.gitignore` - list of ignored files for Git

# Virtual Environment Management

## Quick Examples

```bash
# Create virtual environment in .venv folder (most common)
uv venv

# Create virtual environment in a different folder
uv venv my_project_env

# Create environment using Python 3.11
uv venv --python=3.11 .venv

# Delete and recreate existing environment
uv venv --reset .venv
```

## Common Options

- `--python=3.11` - Choose which Python version to use
- `--reset` - Start fresh by removing old environment first
- `--quiet` - Show less output in terminal
- `--system-site-packages` - Allow using packages installed on your computer

## What Happens Under the Hood

When you run `uv venv`, these steps happen automatically:

1. **Picks a Location**
   - Uses `.venv` folder if you don't specify another
   - Creates new folder if it doesn't exist
   - Cleans up old environment if you use `--reset`

2. **Chooses Python Version**
   Looks for Python in this order:
   1. Version you specified with `--python`
   2. Version from your project's `pyproject.toml` file
   3. Default Python on your computer

3. **Sets Up Environment**
   - Creates a lightweight environment
   - Links to existing Python installation (saves space)
   - Sets up package management tools

## Structure

Your virtual environment will have:
- Python program
- Tools for installing packages
- A separate space for project packages
- Scripts to activate the environment

This setup keeps your project's packages separate from other projects, preventing conflicts.

# Shell Integration

Instead of typing the full `source .venv/bin/activate` command each time, you can set up uv's shell integration.

## Setup

Run once:
```bash
uv venv add-to-path
```

## Usage

After setup, activate any project's virtual environment by running:

```bash
.     # yes, only type dot
```
or 

```bash
source .
```

## How It Works

The `uv venv add-to-path` command modifies your shell configuration:

1. Shell Configuration
    - Modifies your `.bashrc` or `.zshrc`
    - Adds custom function for virtual environment detection
2. Activation Process
- When you run `.` or `source .`, it:
  - Looks for virtual environment in current directory
  - Activates it if found
  - Falls back to normal source behavior if not found

Here's what gets added to your shell configuration:
```bash
# Added to your shell configuration
_uv_venv_source() {
    if [ "$1" = "." ]; then
        if [ -f ".venv/bin/activate" ]; then
            source ".venv/bin/activate"
            return
        fi
    fi
    builtin source "$@"
}
alias .='_uv_venv_source .'
```

## Benefits
- ✓ Simple `.` command works in any project
- ✓ Automatic virtual environment detection
- ✓ Works with bash and zsh
- ✓ Falls back to normal behavior when no environment exists

# Manage dependencies

`uv` uses python builtin env management library `venv`.

* That means `.venv` folder contains your project's virtual environment
* This is where uv will install your project's dependencies.

## Add dependencies

* `uv add requests`
    * adds **latest** compatible version of package: `requests` to the project
    * What happend under the hood:
        1. Updates `pyproject.toml` with new dependency
        2. Automatically regenerates `uv.lock` with exact versions
        3. Installs the package and its dependencies into your virtual environment
        4. Maintains compatibility with existing dependencies
* `uv add 'requests==2.31.0'`
    * adds **specific** version of package: `requests`
    * Same workflow as above, but pins exact version

Common options:

* `uv add requests flask` - add multiple packages at once
* `uv add 'requests>=2.31.0'` - add with version constraint
* `uv add requests --dev` - add as development dependency

What happens behind the scenes:

1. **Config Updates:**
    * Adds dependency to `pyproject.toml`
    * Updates dependency specification (version, extras, etc.)
    * For dev dependencies, adds to `[project.optional-dependencies]`
2. **Lock File Management:**
    * Automatically runs equivalent of `uv lock`
    * Resolves all dependencies (direct and indirect)
    * Updates `uv.lock` with exact versions
3. **Environment Sync:**
    * Automatically runs equivalent of `uv sync`
    * Installs new packages in virtual environment
    * Updates existing packages if needed
4. **Verification:**
    * Checks compatibility with existing dependencies
    * Validates version constraints
    * Ensures reproducible builds

## Update dependencies

* `uv lock --upgrade`
    * upgrades all packages in `uv.lock` file
* `uv lock --upgrade-package requests`
    * tries to update the specified package to the latest compatible version, while keeping the rest of the `uv.lock` file intact 

## Remove dependencies

* `uv remove requests`
    * removes package: `requests`

# Run

## Python script

* `uv run example.py`
    * Runs Python file `example.py` in the project's virtual environment
    * Ensures all dependencies are available before execution
    * Handles environment activation automatically

**What happens when you run this command:**

1. **Environment Check & Setup:**
    * Verifies if virtual environment exists
    * Creates it if missing (equivalent to `python -m venv .venv`)
    * Uses Python version from `.python-version` if available
2. **Dependency Management:**
    * Checks if environment matches `pyproject.toml` and `uv.lock`
    * Installs or updates missing/outdated packages
    * Ensures all dependencies are satisfied
3. **Environment Activation:**
    * Automatically activates the virtual environment
    * Sets up proper Python path and environment variables
    * No manual `source`/`activate` needed
4. **Script Execution:**
    * Runs the specified Python script
    * Passes any additional arguments to the script
    * Maintains proper environment context

**Common Options:**
* `uv run example.py --arg1 value1` - pass arguments to script
* `uv run -m pytest` - run a module (like pytest)
* `uv run --env-file .env example.py` - load environment variables
* `uv run --python 3.11 example.py` - specify Python version

**Equivalent Manual Steps:**

| Command | Description |
|:--------|:------------|
| `uv sync` | Synchronizes python environment with `pyproject.toml` and `uv.lock` configs |
| `source .venv/Scripts/activate` (Unix) or `.venv\Scripts\activate` (Windows) | Activates virtual environment |
| `python example.py` | Run python script |
* Consistent execution environment

# Tools

Detailed docs: https://docs.astral.sh/uv/guides/tools/

* Many Python packages provide applications that can be used as tools.
* uv has specialized support for easily invoking and installing tools.

## Run tool

* `uv tool run ruff`
* `uvx ruff` - preferred shortened variant (`uvx` is a convenient alias for `uv tool run`)

This is advanced version, that allow specify exact version of python package containing tool:

* `uvx ruff@latest check` - latest version of ruff
* `uvx ruff@0.3.0 check` - specific version ruff_v0.3.0

## Install tool

If a tool is used often, it is useful to install it to a persistent environment and add it to the PATH instead of invoking uvx repeatedly.

* `uv tool install ruff`
    *  Installs latest version of tool `ruff`.
    *  This allow running tool `ruff`, but it is not installed package, that could be imported in python.
* `uv tool install ruff>=0.4`
    * Installs specific version of tool `ruff`

## Uninstall tool

* `uv tool uninstall ruff` - uninstall a tool ruff

## Upgrade tool

* `uv tool upgrade ruff` - upgrades tool ruff to latest version
* `uv tool upgrade --all` - upgrades all tools to latest version

## See all tools

* `uv tool list`

## Register tools to shell

* `uv tool update-shell` - Update the shell to include tool executables

# Other commands

## sync

*  `uv sync`
   * Synchronizes your virtual environment to match the project's configuration files
   * Think of it as a one-way sync: your config files (like pyproject.toml) are the source of truth, and `uv sync` updates the environment to match them exactly
   * When you manually edit dependencies in `pyproject.toml`, running `uv sync` will:
       * Add/remove/update packages to match your configuration
       * Rebuild the dependency tree properly
       * Ensure all version constraints are satisfied

## lock

The `uv lock` command manages your project's dependencies through two key files:

* `pyproject.toml` - Your project's direct dependencies
* `uv.lock` - A complete, version-locked dependency tree (contains fixed versions of all direct + indirect dependencies)

Think of `pyproject.toml` as a recipe that lists main ingredients:

```toml
dependencies = [
    "requests>=2.28.0",
    "flask>=2.0.0"
]
```

**When you run** `uv lock`, it:
1. Reads your base dependencies from `pyproject.toml`
2. Resolves the complete dependency tree (including dependencies of dependencies)
3. Determines compatible versions for all packages
4. Writes exact versions to `uv.lock`

For example, your `uv.lock` will include precise versions not just for `requests` and `flask`, but also for their dependencies like `urllib3` and `werkzeug`. This ensures consistent environments across all installations.

**You should run** `uv lock` when:

* Adding or modifying dependencies in `pyproject.toml`
* Updating packages while maintaining version constraints
* Preparing to commit changes that need reproducible builds

Note: While `uv lock` creates the dependency plan, use `uv sync` to actually install the packages in your environment.

## tree

The `uv tree` command displays a visual representation of your project's dependency tree.

When you run `uv tree`, it shows:
* All installed packages in your environment
* Their dependencies indented beneath them
* Version numbers for each package
* Optional markers like development dependencies [dev]

For example:

```
flask==3.0.0
├── blinker==1.7.0
├── click==8.1.7
├── itsdangerous==2.1.2
├── jinja2==3.1.3
│   └── markupsafe==2.1.5
└── werkzeug==3.0.1

requests==2.31.0
├── certifi==2024.2.2
├── charset-normalizer==3.3.2
├── idna==3.6
└── urllib3==2.2.0
```

You can enhance the output with flags:
* `uv tree --with-dev` - Include development dependencies
* `uv tree <package>` - Show the tree for a specific package
* `uv tree --json` - Output in JSON format for programmatic use

This command is useful when:
* Understanding package relationships in your project
* Debugging dependency conflicts
* Identifying which top-level packages bring in specific dependencies
* Auditing your environment's package structure

Note: Unlike `uv lock` which works with planned dependencies, `uv tree` shows currently installed packages in your environment.

# Env variables

Detailed docs: https://docs.astral.sh/uv/configuration/files/#env

`uv` run can load environment variables from dotenv files (`.env` files).

2 options:

* `uv run --env-file .env example.py` -see parameter `--env-file .env`, which loads env variables from `.env` file
* set the `UV_ENV_FILE` environment variable to load a `.env` file from any dedicated location